In [2]:
import numpy as np
import pandas as pd
import tensorflow

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Nadam

np.random.seed(0)

In [3]:
#!pip install tf-nightly

In [4]:
path = "D:/NEU/EAI6000/split-garbage-dataset/split-garbage-dataset"

In [5]:
train_data = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 20,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        horizontal_flip = True,
        vertical_flip = True,
        fill_mode='nearest'
)
validation_data = ImageDataGenerator(
        rescale = 1./255
)
test_data = ImageDataGenerator(
        rescale = 1./255
)

In [6]:
set_shape = (200, 200, 3)

train_batch_size = 256
val_batch_size = 16

train_generator = train_data.flow_from_directory(
            path + '/train',
            target_size = (set_shape[0], set_shape[1]),
            batch_size = train_batch_size,
            class_mode = 'categorical',)

validation_generator = validation_data.flow_from_directory(
            path + '/valid',
            target_size = (set_shape[0], set_shape[1]),
            batch_size = val_batch_size,
            class_mode = 'categorical',
            shuffle=False)

test_generator = test_data.flow_from_directory(
            path + '/test',
            target_size = (set_shape[0], set_shape[1]),
            batch_size = val_batch_size,
            class_mode = 'categorical',
            shuffle=False,)

Found 1768 images belonging to 6 classes.
Found 328 images belonging to 6 classes.
Found 431 images belonging to 6 classes.


In [7]:
vgg = VGG16(weights = 'imagenet', include_top = False,input_shape = set_shape)

In [8]:
for layer in vgg.layers[:-3]:
    layer.trainable = False

In [9]:
model = Sequential()

model.add(vgg)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='sigmoid'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 6, 6, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              18875392  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 6150      
Total params: 33,596,230
Trainable params: 23,601,158
Non-trainable params: 9,995,072
_________________________________________________________________


In [11]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Nadam(lr=1e-4),
              metrics=['acc'])

In [12]:
# Train the model
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('VGG16 Garbage Classifier.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size ,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size,
    verbose=0,
    callbacks = [es, mc],)

Instructions for updating:
Please use Model.fit, which supports generators.

Epoch 00001: val_acc improved from -inf to 0.46951, saving model to VGG16 Garbage Classifier.h5

Epoch 00002: val_acc improved from 0.46951 to 0.58232, saving model to VGG16 Garbage Classifier.h5

Epoch 00003: val_acc improved from 0.58232 to 0.61585, saving model to VGG16 Garbage Classifier.h5

Epoch 00004: val_acc did not improve from 0.61585

Epoch 00005: val_acc improved from 0.61585 to 0.73171, saving model to VGG16 Garbage Classifier.h5

Epoch 00006: val_acc improved from 0.73171 to 0.73476, saving model to VGG16 Garbage Classifier.h5

Epoch 00007: val_acc improved from 0.73476 to 0.74085, saving model to VGG16 Garbage Classifier.h5

Epoch 00008: val_acc did not improve from 0.74085

Epoch 00009: val_acc improved from 0.74085 to 0.76524, saving model to VGG16 Garbage Classifier.h5

Epoch 00010: val_acc improved from 0.76524 to 0.80488, saving model to VGG16 Garbage Classifier.h5

Epoch 00011: val_acc did

In [18]:
test_score = model.evaluate_generator(test_generator, 431/val_batch_size, workers=12)

In [20]:
print("Test_Loss: ", test_score[0], "Test_Accuracy: ", test_score[1])

Test_Loss:  0.6665012836456299 Test_Accuracy:  0.7587006688117981


VGG16 model has a good performance on the train and validation set. However, the performance declined a little on the test set, which reduced the prediction accuracy. It appears that this model has overfitting issue.